<a href="https://colab.research.google.com/github/d-m-bailey/ihp-mpw-be/blob/bandgap_ref_cmos/TO_202504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backend Verification for the iHP Open MPW Shuttles

[D. Mitch Bailey](https://www.linkedin.com/in/mitch-bailey-cvc/), [ShuhariSystem](https://www.shuharisystem.com/)

# Overview
This notebook performs backend LVS (with [klayout](https://github.com/KLayout/klayout) or [magic](https://github.com/RTimothyEdwards/magic)/[netgen](https://github.com/RTimothyEdwards/netgen)), soft-connection checks (with magic/netgen) and/or reliability verfication (with [CVC-RV](https://github.com/d-m-bailey/cvc)) with open source EDA software and systems.

# iHP Open-MPW Data
The 17 submissions to iHp open mpw shuttle for April 2025 listed below are located [here](https://github.com/IHP-GmbH/TO_Apr2025).

* 160GHz_LNA
* 40_GHZ_LOW_NOISE_TIA
* 6502-cpu
* 97_GHZ_LINEAR_TIA
* DC_to_130_GHz_TIA
* GPS_LNA
* Greyhound
* Mixer5GHz
* PA_180GHz
* TTIHP0p2
* TTIHP25a
* VCO_130nm_LSI
* active_L_VCOs
* ascon
* bandgap_ref_cmos
* elemrv-n
* i2c-gpio-expander

Createa a file with the initial environment variables.


In [7]:
%%writefile /content/env
export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025


Overwriting /content/env


# Program installation.
Only needs to be executed once.

This step sets up the pdk and installs magic, klayout, netgen and cvc_rv.

Runtime: 3-4 minutes.

In [8]:
%%shell
cd
cat /content/env
source /content/env

lsb_release -a 2>/dev/null

if [[ ! -d $PDK_ROOT ]]; then
  echo "==> Installing ciel..."
  pip install ciel
fi
ciel enable --pdk $PDK $PDK_COMMIT
# patch for mimcap and antenna error in netgen setup file from pdk
sed -i '/lsearch .cells2/,/circuit2/s/circuit1 .dev/circuit2 $dev/' $PDK_ROOT/$PDK/libs.tech/netgen/ihp-sg13g2_setup.tcl
echo "==> Using pdk $PDK commit $PDK_COMMIT (patched)
"

if [[ ! -d extra_be_checks ]]; then
  echo "==> Downloading extra_be_checks $EXTRA_CHECK_COMMIT"
  rm -rf extra_be_checks
  git clone https://github.com/d-m-bailey/extra_be_checks.git -b $EXTRA_CHECK_COMMIT
fi
echo "==> Using extra_be_checks commit $(cd extra_be_checks; git rev-parse HEAD)
"

if ! command -v netgen; then
  echo "==> Downloading and installing netgen $NETGEN_COMMIT"
  git clone https://github.com/RTimothyEdwards/netgen.git --depth=1 -b $NETGEN_COMMIT
  cd netgen
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using netgen version $(netgen -batch | awk '/Netgen/ {print $2}')
"

if ! command -v magic; then
  echo "==> Downloading and installing magic $MAGIC_COMMIT"
  git clone https://github.com/RTimothyEdwards/magic.git --depth=1 -b $MAGIC_COMMIT
  cd magic
  ./configure --prefix=$LOCAL_INSTALL
  make
  make install
  cd
fi
echo "==> Using magic version $(magic -dnull -noc --version)
"

if ! command -v klayout; then
  echo "==> Downloading and installing klayout $KLAYOUT_DEB for $KLAYOUT_DISTRO"
  wget -P /root https://www.klayout.org/downloads/$KLAYOUT_DISTRO/$KLAYOUT_DEB
  sudo apt install /root/$KLAYOUT_DEB
  cd
fi
echo "==> Using $(klayout -v)
"

if ! command -v cvc_rv; then
  echo "==> Downloading and installing cvc_rv $CVC_COMMIT"
  sudo apt install autopoint bison flex
  git clone https://github.com/d-m-bailey/cvc --depth=1 -b $CVC_COMMIT
  cd cvc
  autoreconf -vif
  ./configure --prefix=$LOCAL_INSTALL --disable-nls
  make
  make install
  cd
fi
echo "==> Using $(cvc_rv -v)
"

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy
⠋ Enabling version ddb601a4a4473163e1ed6df416b885df18b4ac03…
Version ddb601a4a4473163e1ed6df416b885df18b4ac03 enabled for the ihp-sg13g2 PDK.
==> Using pdk ihp-sg13g2 commit ddb601a4a4473163e1ed6df416b885df18b4ac03 (patched)

==> Using extra_be_checks commit 895a40f5ba4aee557bdeb74f49836a757a3bba7f

/content/local/bin/netgen
==> Using netgen version 1.5.294

/content/local/bin/magic
==> Using magic version 8.3.

# Clone the design repo and list the gds, spice and verilog files.

Run to refresh the design data.
The file list is also saved in /content/filelist.txt.

Runtime: 1-2 minutes.

In [3]:
%%shell
cat /content/env
source /content/env
rm -rf data
git clone https://github.com/IHP-GmbH/$MPW.git data
cd data
# uncompress gds, spice, and verilog files
find . \( -name "*.gds.gz" -o -name "*.spice.gz" -o -name "*.v.gz" \) -execdir gunzip -v {} \;
find . \( -name "*.gds.zip" -o -name "*.spice.zip" -o -name "*.v.zip" \) -execdir 7z x {} \;
# Create a file listing the size, project and gds file name.
# Only list files that end with .gds (in gds subdirectories), .spice, or .v.
tee /content/filelist.txt <<EOF
   Size Project                   File
------- ------------------------- ---------------------------------------------
EOF
ls -s $(find . \( -name "*.gds" -path "*/gds/*" \) -o -name "*.spice" -o -name "*.v") |
  sed -e 's,\./,,' -e 's,/, ,' |
  awk '{printf "%7d %-25s %s\n", $1, $2, $3}' |
  tee -a /content/filelist.txt

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks

export MPW=TO_Apr2025
unset WORK_ROOT
Cloning into 'data'...
remote: Enumerating objects: 3270, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 3270 (delta 154), reused 149 (delta 143), pack-reused 3095 (from 2)
Receiving objects: 100% (3270/3270), 784.55 MiB | 28.29 MiB/s, done.
Resolving deltas: 100% (1503/1503), done.
Updating files: 100% (886/886), done.
./FMD_QNC_11_SG13G2Top.gds.gz:	 84.8% -- replaced with ./FMD_QNC_11_SG13G2Top.gds
./FMD_QNC

# Set the project, top cell and gds file names.

In [4]:
%%shell
# remove existing values before adding new ones
sed -i -e '/^export PROJECT=/d' \
  -e '/^export LAYOUT_TOP=/d' \
  -e '/^export GDSFILE=/d' \
  /content/env
cat >> /content/env <<EOF
export PROJECT=bandgap_ref_cmos
export LAYOUT_TOP=full_bandgap
export GDSFILE=design_data/gds/FMD_QNC_15_WeakInvBGR.gds
EOF

# Create setup for bandgap_ref_cmos

In [12]:
%%writefile /content/data/bandgap_reference.spice
** sch_path: /Users/home/IHP/TO_Apr2025/bandgap_ref_cmos/design_data/xschem/part_2_full_bgr/bandgap_reference.sch
.subckt bandgap_reference vplus v- Vo1 VBG GND
*.PININFO vplus:B v-:B Vo1:B VBG:O
XM8 net1 v- GND GND sg13_lv_nmos l=10u w=150n ng=1 m=1
XM6 net1 net1 vdd vdd sg13_lv_pmos l=1u w=1u ng=1 m=1
XM7 net2 net1 vdd vdd sg13_lv_pmos l=1u w=1u ng=1 m=1
XM9 v- net2 vdd vdd sg13_lv_pmos l=4u w=200n ng=1 m=1
XM1 v- vplus GND GND sg13_lv_nmos l=5u w=7.14u ng=4 m=1
XM2 net3 net3 GND GND sg13_lv_nmos l=5u w=21u ng=8 m=1
XM3 v- Vo1 vdd vdd sg13_lv_pmos l=5u w=15u ng=8 m=1
XM4 vplus Vo1 vdd vdd sg13_lv_pmos l=5u w=15u ng=8 m=1
XM5 VBG Vo1 vdd vdd sg13_lv_pmos l=5u w=16u ng=8 m=1
XC3 VBG GND cap_cmim w=72.965e-6 l=72.965e-6 m=1
XR3 net3 vplus rppd w=0.5e-6 l=194.345e-6 m=1 b=0
XR1 GND vplus rppd w=0.6e-6 l=194.345e-6 m=1 b=0
XR2 GND VBG rppd w=0.5e-6 l=192.395e-6 m=1 b=0
XC1 net2 GND cap_cmim w=18.195e-6 l=18.195e-6 m=1
x1 vdd GND vplus v- Vo1 GND two_stage_OTA
.ends

* expanding   symbol:  part_1_OTA/two_stage_OTA.sym # of pins=6
** sym_path: /Users/home/IHP/TO_Apr2025/bandgap_ref_cmos/design_data/xschem/part_1_OTA/two_stage_OTA.sym
** sch_path: /Users/home/IHP/TO_Apr2025/bandgap_ref_cmos/design_data/xschem/part_1_OTA/two_stage_OTA.sch
.subckt two_stage_OTA vdd iout vplus v- vout vss
*.PININFO v-:B vplus:B vss:B vdd:B iout:B vout:B
XM4 net3 net1 vss vss sg13_lv_nmos l=9.75u w=720n ng=1 m=1
XM3 net1 net1 vss vss sg13_lv_nmos l=9.75u w=720n ng=1 m=1
XM1 net1 v- net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM2 net3 vplus net2 vdd sg13_lv_pmos l=3.7u w=3.64u ng=1 m=2
XM5 net2 iout vdd vdd sg13_lv_pmos l=1.95u w=5.3u ng=1 m=1
XM7 vout iout vdd vdd sg13_lv_pmos l=2.08u w=75u ng=8 m=1
XM6 vout net3 vss vss sg13_lv_nmos l=9.75u w=28.8u ng=4 m=1
XM9 iout iout vdd vdd sg13_lv_pmos l=2.08u w=75u ng=8 m=1
XC2 net3 vout cap_cmim w=22.295e-6 l=22.295e-6 m=1
.ends


Overwriting /content/data/bandgap_reference.spice


Create lvs_config.json

Update the TOP_LAYOUT, LVS_SPICE_FILES, LVS_VERILOG_FILES, and LAYOUT_FILE for every design.

Update the other parameters as needed.

In [16]:
%%writefile /content/lvs_config.json
{
  "PROJECT": "bandgap_ref_cmos",
  "#STD_CELL_LIBRARY": "sky130_fd_sc_hd",
  "#INCLUDE_CONFIGS": [
    "$LVS_ROOT/tech/$PDK/lvs_config.base.json"
  ],
  "TOP_SOURCE": "bandgap_reference",
  "TOP_LAYOUT": "full_bandgap",
  "EXTRACT_FLATGLOB": [ "" ],
  "EXTRACT_ABSTRACT": [ "*FILL_CELL*" ],
  "LVS_FLATTEN": [ "" ],
  "LVS_NOFLATTEN": [ "" ],
  "LVS_IGNORE": [ "*FILL_CELL*" ],
  "LVS_SPICE_FILES": [
    "/content/data/bandgap_reference.spice"
  ],
  "#LVS_VERILOG_FILES": [
    "$UPRJ_ROOT/verilog/gl/user_proj_example.v",
    "$UPRJ_ROOT/verilog/gl/user_project_wrapper.v"
  ],
  "LAYOUT_FILE": "$UPRJ_ROOT/$PROJECT/design_data/gds/FMD_QNC_15_WeakInvBGR.gds"
}

Overwriting /content/lvs_config.json


# Run BE checks

In [17]:
%%shell
cat /content/env
source /content/env
cd data
export WORK_ROOT=$PWD/work
rm -rf $WORK_ROOT
mkdir $WORK_ROOT
$LVS_ROOT/run_be_checks /content/lvs_config.json

export LOCAL_INSTALL=/content/local
export PATH=$PATH:$LOCAL_INSTALL/bin
export PDK_ROOT=/content/pdks
export PDK=ihp-sg13g2
export PDKPATH=$PDK_ROOT/$PDK
export PDK_COMMIT=ddb601a4a4473163e1ed6df416b885df18b4ac03
export MAGIC_COMMIT=master
export NETGEN_COMMIT=master
export KLAYOUT_DISTRO=Ubuntu-22
export KLAYOUT_DEB=klayout_0.30.1-1_amd64.deb
export EXTRA_CHECK_COMMIT=ihp-sg13g2
export CVC_COMMIT=master
export UPRJ_ROOT=/content/data
export LVS_ROOT=/root/extra_be_checks
export MPW=TO_Apr2025
Loading LVS environment from /content/lvs_config.json

TOP SOURCE: bandgap_reference
SOURCE FILE(S): /content/data/bandgap_reference.spice
TOP LAYOUT: full_bandgap
LAYOUT FILE: /content/data/bandgap_ref_cmos/design_data/gds/FMD_QNC_15_WeakInvBGR.gds
EXTRACT_FLATGLOB: 
EXTRACT_ABSTRACT: *FILL_CELL*
LVS_FLATTEN: 
LVS_NOFLATTEN: 
LVS_IGNORE: *FILL_CELL*
WORK_ROOT   : /content/data/work
LOG_ROOT    : /content/data/work
SIGNOFF_ROOT: /content/data/work
 
Running hierarchical comparison between verilo

CalledProcessError: Command 'cat /content/env
source /content/env
cd data
export WORK_ROOT=$PWD/work
rm -rf $WORK_ROOT
mkdir $WORK_ROOT
$LVS_ROOT/run_be_checks /content/lvs_config.json
' returned non-zero exit status 1.

# View verification logs

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.log')
files.view('/content/data/work/lvs.log')
files.view('/content/data/work/cvc.log')
files.view('/content/data/work/cvc.oeb.log')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# View soft verification report

In [ ]:
from google.colab import files

files.view('/content/data/work/soft.report')

<IPython.core.display.Javascript object>

# View LVS report

In [ ]:
from google.colab import files

files.view('/content/data/work/lvs.report')

<IPython.core.display.Javascript object>

# View CVC report

In [ ]:
from google.colab import files

files.view('/content/data/work/cvc.error')

<IPython.core.display.Javascript object>

# View OEB report

In [ ]:
from google.colab import files

files.view('/content/data/work/cvc.oeb.sort.report')

<IPython.core.display.Javascript object>